In [1]:
%matplotlib inline

In [2]:
# Import relevant libraries

import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns
import math
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
from statsmodels.api import add_constant
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as sm2
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from dmba import classificationSummary
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    f1_score,
    confusion_matrix, 
    precision_recall_fscore_support,
    roc_curve, 
    accuracy_score, 
    roc_auc_score)

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/2536382723.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


no display found. Using non-interactive Agg backend


In [3]:
df = pd.read_csv('Omni_98_20.csv')

In [4]:
df.info() # check general information about the dataset.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289296 entries, 0 to 289295
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   time     289296 non-null  object 
 1   Bx       289296 non-null  float64
 2   By       289296 non-null  float64
 3   Bz       289296 non-null  float64
 4   density  289296 non-null  float64
 5   V        289296 non-null  int64  
 6   Dst      289296 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 15.5+ MB


In [5]:
df.describe()

,Bx,By,Bz,density,V,Dst
count,289296.000000,289296.000000,289296.000000,289296.000000,289296.000000,289296.000000
mean,125.451494,125.539402,125.507402,150.690582,1665.224635,-14.351678
std,331.221853,331.327176,331.331903,350.023119,3210.111810,22.262813
min,-40.800000,-43.100000,-57.800000,0.100000,228.000000,-589.000000
25%,-2.200000,-2.100000,-1.300000,3.500000,361.000000,-22.000000
50%,0.800000,0.700000,0.300000,5.800000,424.000000,-10.000000
75%,3.700000,3.700000,2.500000,11.300000,549.000000,-1.000000
max,999.900000,999.900000,999.900000,999.900000,9999.000000,81.000000


In [6]:
df

,time,Bx,By,Bz,density,V,Dst
0,01-Jan-1988 00:00:00,999.9,999.9,999.9,999.9,9999,-16
1,01-Jan-1988 01:00:00,999.9,999.9,999.9,999.9,9999,-10
2,01-Jan-1988 02:00:00,999.9,999.9,999.9,999.9,9999,-8
3,01-Jan-1988 03:00:00,4.1,-1.9,-0.5,14.6,282,-6
4,01-Jan-1988 04:00:00,5.3,-2.0,0.3,12.5,283,-6
...,...,...,...,...,...,...,...
289291,31-Dec-2020 19:00:00,-0.7,-1.7,0.9,4.0,385,3
289292,31-Dec-2020 20:00:00,-0.6,-2.0,0.7,4.4,378,6
289293,31-Dec-2020 21:00:00,-0.1,-2.0,-0.2,4.4,373,4
289294,31-Dec-2020 22:00:00,2.3,-0.8,-0.4,4.4,363,3


In [7]:
# dataset has lots of missing values, are represented by 999.9 in Bx, By, Bz columns and 9999 in V column. These missing 
# values will substentially impact the further result of analysis. The quantity of missing values should be discovered
# to understand how many missing values there are.

print('Number of missing values for Bx:', df["Bx"].value_counts()[999.9], '\nproportion of missing values to total number of records', round(df["Bx"].value_counts()[999.9]/len(df['Bx']), 4) * 100, '%')
print('\nNumber of missing values for By:', df["By"].value_counts()[999.9], '\nproportion of missing values to total number of records', round(df["By"].value_counts()[999.9]/len(df['By']), 4) * 100, '%')
print('\nNumber of missing values for Bz:', df["Bz"].value_counts()[999.9], '\nproportion of missing values to total number of records', round(df["Bz"].value_counts()[999.9]/len(df['Bz']), 4) * 100, '%')
print('\nNumber of missing values for density:', df["density"].value_counts()[999.9], '\nproportion of missing values to total number of records', round(df["density"].value_counts()[999.9]/len(df['density']), 4) * 100, '%')
print('\nNumber of missing values for V:', df["V"].value_counts()[9999], '\nproportion of missing values to total number of records', round(df["V"].value_counts()[9999]/len(df['V']), 4) * 100, '%')

Number of missing values for Bx: 36295 
proportion of missing values to total number of records 12.55 %

Number of missing values for By: 36321 
proportion of missing values to total number of records 12.55 %

Number of missing values for Bz: 36321 
proportion of missing values to total number of records 12.55 %

Number of missing values for density: 42004 
proportion of missing values to total number of records 14.52 %

Number of missing values for V: 37349 
proportion of missing values to total number of records 12.91 %


In [8]:
# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Convert missing values to NaN
df.replace({999.9: pd.NA, 9999: pd.NA, 99999: pd.NA}, inplace=True)

# Group by year and count missing values for each column
missing_values_by_year = df.groupby(df['time'].dt.year).apply(lambda x: x.isna().sum())

# Plot histogram for each column
for column in df.columns:
    if column != 'time':  # Exclude the 'time' column from plotting
        plt.figure()
        missing_values_by_year[column].plot(kind='bar', title=f'Missing Values for {column} by Year')
        plt.xlabel('Year')
        plt.ylabel('Number of Missing Values')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.legend()
plt.show()


/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/610046424.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
missing_values_by_year = df.groupby(df['time'].dt.year).apply(lambda x: x.isna().sum()/len(df[column]) * 100)

for column in df.columns:
    if column != 'time':  # Exclude the 'time' column from plotting
        plt.figure()
        missing_values_by_year[column].plot(kind='bar', title=f'{column} missing values by year')
        plt.xlabel('Year')
        plt.ylabel('Number of Missing Values (in %)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.legend()
plt.show()

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/190392829.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# As can be seen, missing values for most of the variables are located in the timespan between 1988 and 1984. 
# The missing data is classified as Missing At Random (MAR), as the reason for the missing values is not know,
# but there is almost perfect correlation for missing values in the dataset, except for density values.
# The reason is likely to be the faultiness of the measurement system. 

# Because the most of the missing values are highly correlated it can be completely removed. However, there will be some missing values
# present in density column which will impact the result of the dataset. 

# The missing values will be dropped. 
df = df.dropna()

In [11]:
binsL = 20

colors = ["red", "orange", "navy", "green", "blue", "purple", "black", "magenta"] # different colors for different plots
units = ['nT', 'nT', 'nT', 'N/cm^3', 'km/s', 'nT']

# Create a new figure and subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 10)) # set the number of rows, columns and figure size
axes = axes.flatten() # Flatten the axes array for easy iteration

for i, metric in enumerate(df.columns[1:]): # exclude time column
    ax = axes[i]  # Select the current axis
    c = colors[i] # Assign the color
    ax.hist(df[metric], bins=binsL, label=metric, linewidth=1.5, color=c, linestyle='-', histtype='step')
    ax.legend()  # Add legend for each histogram
    ax.set_title('Distribution of {}'.format(metric))
    ax.set_xlabel('{}'.format(units[i]))
    ax.set_ylabel('Counts')

for i in range(len(df.columns), 6):  # Remove any extra subplots
    fig.delaxes(axes[i])

plt.tight_layout() # Adjust layout to prevent overlap
plt.show()
plt.savefig('Histograms_of_metrics')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3940030398.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
mean_data = df[1:].mean() # extract the mean, median, mode and standard deviation data
median_data = df[1:].median()
mode_data = df[1:].mode()
std_data =  df[1:].std()
quantile_data = df[1:].quantile(q = [0.25, 0.5, 0.75], axis = 0) # extract the quartile data
iqr_data = quantile_data.iloc[2] - quantile_data.iloc[0] # find the interquatile range by subtracting third quartile (75%) from the first quartile
df_val_table = pd.DataFrame(
    {
        'Mean': mean_data,
        'Median': median_data,
        'Mode': mode_data.iloc[0],
        'Standard Deviation': std_data,
        '1st Quartile (25%)': quantile_data.iloc[0],
        '2nd Quartile (50%)': quantile_data.iloc[1],
        '3rd Quartile (75%)': quantile_data.iloc[2],
        'Interquantile Range (IQR)': iqr_data
}      
)

df_val_table.transpose().drop(columns = ['time'])

,Bx,By,Bz,density,V,Dst
Mean,0.008291,0.003223,-0.033854,6.442488,431.556273,-12.990458
Median,0.0,0.0,0.0,5.0,408.0,-9.0
Mode,-2.7,-2.1,0.0,2.7,376,-4.0
Standard Deviation,3.523015,3.828936,3.015002,5.190981,102.117503,20.274161
1st Quartile (25%),-2.6,-2.4,-1.5,3.2,356.0,-21.0
2nd Quartile (50%),0.0,0.0,0.0,5.0,408.0,-9.0
3rd Quartile (75%),2.5,2.4,1.4,8.0,486.0,-1.0
Interquantile Range (IQR),5.1,4.8,2.9,4.8,130.0,20.0


In [13]:
df.describe()

,time,Dst
count,246486,246486.000000
mean,2006-06-29 02:13:23.539348992,-12.990429
min,1988-01-01 03:00:00,-422.000000
25%,1999-05-26 00:15:00,-21.000000
50%,2006-11-01 12:30:00,-9.000000
75%,2013-12-02 02:45:00,-1.000000
max,2020-12-31 23:00:00,77.000000
std,NaN,20.274125


In [14]:
plt.figure(figsize = (14, 10))
mask = np.triu(np.ones_like(df.corr()))
sns.heatmap(df.corr(), cmap = 'RdYlBu', mask = mask, annot=True, fmt=".2f", linewidths=.5)
plt.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4165932874.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [15]:
df_no_time = df.drop(columns = ['time'])
df_no_time.corr()
# most of the values have really low correlation between each other. Highest correlation is between Dst and Bz
# Highest negative correlation is between the Dst and speed of solar wind, By and Bx.

,Bx,By,Bz,density,V,Dst
Bx,1.000000,-0.441654,0.000437,0.009540,-0.010394,-0.012527
By,-0.441654,1.000000,0.007375,-0.000208,0.026367,-0.011702
Bz,0.000437,0.007375,1.000000,0.018452,-0.015626,0.304729
density,0.009540,-0.000208,0.018452,1.000000,-0.355335,0.219648
V,-0.010394,0.026367,-0.015626,-0.355335,1.000000,-0.472186
Dst,-0.012527,-0.011702,0.304729,0.219648,-0.472186,1.000000


In [16]:
# Iterate through all columns in the DataFrame
for i, col1 in enumerate(df_no_time.columns):
    for j, col2 in enumerate(df_no_time.columns[i+1:]):
        plt.figure(figsize=(8, 6))
        plt.hist2d(df_no_time[col1], df_no_time[col2], cmap='viridis', bins=120)
        plt.xlabel(col1)
        plt.ylabel(col2)
        plt.title(f'2D Histogram: {col1} vs {col2}')
        plt.colorbar()
        plt.show()

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1993153407.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1993153407.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))


In [17]:
df['log_Dst'] = np.log10(abs(df['Dst'])) # Because logarithm of negative values does'nt exist, use absolute value
df_copy = df.copy() # create a copy of the dataset

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1879289792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['log_Dst'] = np.log10(abs(df['Dst'])) # Because logarithm of negative values does'nt exist, use absolute value


In [18]:
column1 = 'Dst'
column2 = 'log_Dst'
colnames = df.columns.tolist()
index1, index2 = colnames.index(column1), colnames.index(column2) # extract indices from chosen columns
colnames[index2], colnames[index1] = colnames[index1], colnames[index2] # change the indices to swap columns places
df = df[colnames] 

df['log_Dst'] = df_copy['log_Dst'] # swap columns places using columns from copied dataset
df['Dst'] = df_copy['Dst']

df

,time,Bx,By,Bz,density,V,log_Dst,Dst
3,1988-01-01 03:00:00,4.1,-1.9,-0.5,14.6,282,0.778151,-6
4,1988-01-01 04:00:00,5.3,-2.0,0.3,12.5,283,0.778151,-6
5,1988-01-01 05:00:00,4.2,-3.8,0.1,16.3,282,0.477121,-3
6,1988-01-01 06:00:00,3.5,-4.4,0.4,18.2,284,0.602060,-4
7,1988-01-01 07:00:00,0.7,-2.0,2.0,20.5,296,0.602060,-4
...,...,...,...,...,...,...,...,...
289291,2020-12-31 19:00:00,-0.7,-1.7,0.9,4.0,385,0.477121,3
289292,2020-12-31 20:00:00,-0.6,-2.0,0.7,4.4,378,0.778151,6
289293,2020-12-31 21:00:00,-0.1,-2.0,-0.2,4.4,373,0.602060,4
289294,2020-12-31 22:00:00,2.3,-0.8,-0.4,4.4,363,0.477121,3


In [19]:
time_lag = 1000
# calculate autocorrelations at timelag in range of 1 - 1000 hours for every column
for metric in list(df_no_time.columns): 
    print(f'Autocorrelation of {metric}, using timelag of {time_lag}:', sm.tsa.acf(df_no_time[metric], nlags=time_lag), '\n') 

Autocorrelation of Bx, using timelag of 1000: [ 1.          0.86585726  0.76454472 ... -0.05107561 -0.05081353
 -0.05068413] 

Autocorrelation of By, using timelag of 1000: [ 1.          0.84512926  0.72931095 ... -0.02699586 -0.02844526
 -0.02849462] 

Autocorrelation of Bz, using timelag of 1000: [ 1.          0.71920777  0.52799486 ... -0.00297874 -0.00394501
 -0.00376307] 

Autocorrelation of density, using timelag of 1000: [1.         0.91809363 0.82844712 ... 0.03353293 0.03345532 0.03316404] 

Autocorrelation of V, using timelag of 1000: [1.         0.99046509 0.97976152 ... 0.09275435 0.09284675 0.09286461] 

Autocorrelation of Dst, using timelag of 1000: [1.         0.97312032 0.93211277 ... 0.07586083 0.0763142  0.07664375] 



In [20]:
time_lag = 1000
colors = ["red", "orange", "navy", "green", "blue", "purple", "black", "magenta"] # different colors for different plots

for i, metric in enumerate(df_no_time.columns): 
    plt.figure(figsize = (16, 10))
    tsaplots.plot_acf(df_no_time[metric], lags = time_lag, color = colors[i])
    plt.title(f'Autocorrelation of {metric} with a time lag in range of 1 to {time_lag} hours')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [21]:
# Fit the linear regression model on the whole dataset, as in regression there are no testing and training dataset, regression is fit to a all of the data.

X_ = df[["Bx","By","Bz","density","V"]]
y_ = df['Dst']

X_ = sm.add_constant(X_)
est= sm.OLS(y_, X_.astype(float)).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                    Dst   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                 2.261e+04
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        16:46:11   Log-Likelihood:            -1.0450e+06
No. Observations:              246486   AIC:                         2.090e+06
Df Residuals:                  246480   BIC:                         2.090e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.0576      0.177    135.706      0.0

In [22]:
df_88_19 = df[df['time'].dt.year <= 2019] # create two separate datasets for training and testing, with data from 1988-2019 and 2020 respectively
df_20 = df[df['time'].dt.year >= 2020]

np.asarray(df_88_19) # convert to an array
np.asarray(df_20)

array([[Timestamp('2020-01-01 00:00:00'), 0.3, -0.6, ..., 295,
        0.7781512503836436, -6],
       [Timestamp('2020-01-01 01:00:00'), 2.2, 0.4, ..., 299, -inf, 0],
       [Timestamp('2020-01-01 02:00:00'), 3.3, -1.6, ..., 300,
        0.6020599913279624, 4],
       ...,
       [Timestamp('2020-12-31 21:00:00'), -0.1, -2.0, ..., 373,
        0.6020599913279624, 4],
       [Timestamp('2020-12-31 22:00:00'), 2.3, -0.8, ..., 363,
        0.47712125471966244, 3],
       [Timestamp('2020-12-31 23:00:00'), 1.0, -1.0, ..., 366,
        0.47712125471966244, 3]], dtype=object)

In [23]:
df.loc[33605]

time       1991-11-01 05:00:00
Bx                       -28.7
By                        22.3
Bz                         3.3
density                    3.8
V                          822
log_Dst               1.924279
Dst                        -84
Name: 33605, dtype: object

In [24]:
df.loc[33602]


time       1991-11-01 02:00:00
Bx                       -32.9
By                         8.3
Bz                        22.3
density                   14.8
V                          918
log_Dst               1.880814
Dst                        -76
Name: 33602, dtype: object

In [25]:
sample_size = 28  # Limit sample size 
X_sample = X_.sample(n=sample_size, random_state=42)
y_sample = y_.loc[X_sample.index]

# Fit the model on the sampled data
X_sample = sm.add_constant(X_sample)  # Add constant for the intercept
est_ = sm.OLS(y_sample, X_sample.astype(float)).fit()

influence_plot(est_) # create an influence plot
 
n=df.shape[0]
k=df.shape[1]
leverage_cutoff =3*(k+1)/n
plt.axvline(x=leverage_cutoff,linestyle=':',label='Leverage Cutoff')
plt.legend()
plt.show()
plt.savefig('influence_plt')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1503440196.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [26]:
predictors = ["Bx","By","Bz","density","V"]
outcome = 'Dst'

model = LinearRegression()
model.fit(df[predictors], df[outcome]) # fit linear regression to the data

fitted = model.predict(df_88_19[predictors])
residuals = df_88_19[outcome] - fitted

plt.hist(residuals, bins = 30)
plt.title('Histogram of residual values using data from 1988 to 2019')
plt.xlabel('Residual values')
plt.ylabel('Counts')
plt.xlim(-200, 100) # Set limits for x values in the plot
plt.show()

# The following residual distribution is skewed to the left of the graph.

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/304872244.py:15: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
fitted = model.predict(df_88_19[predictors])
abs_residuals = abs(df_88_19[outcome] - fitted)

plt.scatter(abs_residuals, df_88_19[outcome], color = 'red', s = 2.5) # plot the absolute residual value against the actual values
plt.title('Scatterplot of residual values using data from 1988 - 2019')
plt.xlabel('Absolute residual values')
plt.ylabel('Actual values')
plt.show()

# The following residual distribution is skewed to the left of the graph.

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3355175372.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [28]:
df_88_19

,time,Bx,By,Bz,density,V,log_Dst,Dst
3,1988-01-01 03:00:00,4.1,-1.9,-0.5,14.6,282,0.778151,-6
4,1988-01-01 04:00:00,5.3,-2.0,0.3,12.5,283,0.778151,-6
5,1988-01-01 05:00:00,4.2,-3.8,0.1,16.3,282,0.477121,-3
6,1988-01-01 06:00:00,3.5,-4.4,0.4,18.2,284,0.602060,-4
7,1988-01-01 07:00:00,0.7,-2.0,2.0,20.5,296,0.602060,-4
...,...,...,...,...,...,...,...,...
280507,2019-12-31 19:00:00,-0.5,-0.1,-3.1,11.1,310,0.301030,-2
280508,2019-12-31 20:00:00,1.1,-0.3,-3.9,10.3,308,0.000000,-1
280509,2019-12-31 21:00:00,1.3,-0.4,-2.2,9.9,308,0.000000,-1
280510,2019-12-31 22:00:00,0.7,-0.4,-1.6,9.4,305,0.477121,-3


In [29]:
outcome = 'Dst'

# Fit the linear regression model outside the loop
model = LinearRegression()
X = df_88_19[df_88_19.columns[1:-2]].values
y = df_88_19[outcome].values.reshape(-1, 1)
model.fit(X, y)

# Create the figure
fig = plt.figure(figsize=(10, 7))
for i, metric in enumerate(df_88_19.columns[1:-2]):
    # Predictions and residuals for the current independent variable
    fitted = model.predict(X)
    residuals = y - fitted

    # Create the partial residual plot
    plt.scatter(df_88_19[metric], residuals, color=colors[i], label=metric)
    plt.title(f'Partial Residual Plot of {metric} with respect to {outcome}', size=24)
    plt.xlabel(f'{metric} values', size=18)
    plt.ylabel('Residuals', size=18)
    plt.legend()
    plt.show()


/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4246090351.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4246090351.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4246090351.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4246090351.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/4246090351.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [30]:
# Backward elimination for linear regression
# Duplicate the regression summary for the ease of analysis

print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                    Dst   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                 2.261e+04
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        16:46:21   Log-Likelihood:            -1.0450e+06
No. Observations:              246486   AIC:                         2.090e+06
Df Residuals:                  246480   BIC:                         2.090e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.0576      0.177    135.706      0.0

In [31]:
# As the p-values are all non-zero, try to remove features one by one to check if the key metrics will change,
# such as R-squared (the closer to 1 the better), p-values (below 0.05 is good) for individual variables 
# and F-statistic (the higher the better).

# After trial and error, the best model was chosen to be with "Bz", "density", "V" as features, with R-squared value of 0.312
# p-values of 0 for each variable and F-statistic of 3.596e+04.     

X = df_88_19[["Bz","density","V"]]
y = df_88_19['Dst']

X = sm.add_constant(X)
est_88_19 = sm.OLS(y, X.astype(float)).fit()
print(est_88_19.summary())

                            OLS Regression Results                            
Dep. Variable:                    Dst   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                 3.596e+04
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        16:46:21   Log-Likelihood:            -1.0108e+06
No. Observations:              237758   AIC:                         2.022e+06
Df Residuals:                  237754   BIC:                         2.022e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.9306      0.183    131.058      0.0

In [32]:
# Principal Component Analysis

predictors = ["Bx", "By", "Bz", "density", "V"]

sc = StandardScaler()
X = sc.fit_transform(df[predictors])

pca = PCA(n_components = len(predictors))
pca.fit(X)

PCA(n_components=5)

In [33]:
explained_variance = pd.DataFrame(pca.explained_variance_) # explained variance calculates how much of the total variance in the original dataset is explained by each principal component

ax = explained_variance.head(10).plot.bar(legend = False, figsize = (4, 4))
ax.set_xlabel('Component')
ax.set_ylabel('eigenvalues')
plt.tight_layout()
plt.show()

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1537303561.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [34]:
ev_ratio = pd.DataFrame(pca.explained_variance_ratio_)

evr_sum = pca.explained_variance_ratio_.cumsum()
print('explained_variance_ratio:', ev_ratio.round(3)) # normalize the results
print('explained_variance_ratio_cumsum', evr_sum.round(3)) #compute cumulative sum

explained_variance_ratio:        0
0  0.290
1  0.270
2  0.200
3  0.129
4  0.111
explained_variance_ratio_cumsum [0.29  0.56  0.759 0.889 1.   ]


In [35]:
ax = ev_ratio.head(10).plot.bar(legend = False, figsize = (4, 4))
ax.set_xlabel('Component')
ax.set_ylabel('weight ratio')
plt.tight_layout()
plt.show()

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1302780401.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [36]:
loadings = pd.DataFrame(pca.components_, columns = predictors)
print(loadings.round(3))

# Loadings describe how much each variable contributes to a particular principar component. Basically, they show the 
# correlation between variable and principal component. Negative loading simply means that a certain characteristic is 
# inversely correlated with given principal component. 

      Bx     By     Bz  density      V
0 -0.685  0.686 -0.002   -0.161  0.185
1 -0.175  0.170  0.070    0.687 -0.681
2 -0.020  0.001 -0.997    0.044 -0.052
3  0.112  0.089 -0.008    0.700  0.700
4  0.698  0.702 -0.012   -0.097 -0.104


In [37]:
# Classifier models

df['Dst_label'] = np.where(df['Dst'] < -20, True, False) 
df_88_19['Dst_label'] = np.where(df_88_19['Dst'] < -20, True, False) 
df_20['Dst_label'] = np.where(df_20['Dst'] < -20, True, False) 

# Add a boolean value column indicating whether there is a storm, 
# based on Dst value. If Dst < -20, there is a storm. If Dst >= -20, there is no storm. 

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1787977780.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_88_19['Dst_label'] = np.where(df_88_19['Dst'] < -20, True, False)
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/1787977780.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20['Dst_label'] = np.where(df_20['Dst'] < -20, True, False)


In [38]:
plt.figure(figsize=(5,5))
plt.pie(df_88_19["Dst_label"].value_counts().values,labels=["No magnetic storms","Magnetic storm"], autopct="%1.0f%%")
plt.title("Proportion of target variable in dataset 1988-2019")
plt.show()
# plt.savefig('Pie_chart.png')

print('Magnetic storm?\n', '0 - no (False)\n', '1 - yes (True)\n', df_88_19['Dst_label'].value_counts()) # Check the Dst_label to understand what is the ratio of magnetic storms.
print("Magnetic storm ratio:", (df_88_19['Dst_label'].value_counts()[1]/np.sum(df_88_19['Dst_label'].value_counts())*100).round(2), "%")

Magnetic storm?
 0 - no (False)
 1 - yes (True)
 Dst_label
False    176918
True      60840
Name: count, dtype: int64
Magnetic storm ratio: 25.59 %


/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/360939721.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/360939721.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Magnetic storm ratio:", (df_88_19['Dst_label'].value_counts()[1]/np.sum(df_88_19['Dst_label'].value_counts())*100).round(2), "%")


In [39]:
# There is a 1/4 ratio of magnetic storms to no magnetic storms. The dataset is relatively large, so we can 
# balance the dataset by deleting records from prevalent class, as majority of classification models assume that balanced datasets are used for training purposes.

In [40]:
# Assuming you've already selected the random false values
random_false_values = df_88_19[df_88_19["Dst_label"] == False].sample(frac = 0.65).index

# Drop the selected random false values from the df_88_19 dataset
df_88_19_dropped = df_88_19.drop(random_false_values)

print("Number of elements dropped from no magnetic storm instances:", len(random_false_values))

Number of elements dropped from no magnetic storm instances: 114997


In [41]:
# Check distribution again

plt.figure(figsize=(5,5))
plt.pie(df_88_19_dropped["Dst_label"].value_counts().values,labels=["No magnetic storms","Magnetic storm"], autopct="%1.0f%%")
plt.title("Proportion of target variable in dataset")
plt.show()

print('Magnetic storm?\n', 'False - no\n', 'True - yes\n', df_88_19_dropped['Dst_label'].value_counts()) # Check the Dst_label to understand what is the ratio of magnetic storms.
print("Magnetic storm ratio:", (df_88_19_dropped['Dst_label'].value_counts()[1]/np.sum(df_88_19_dropped['Dst_label'].value_counts())*100).round(2), "%")
print("Total number of elements in training dataset:", (len(df_88_19_dropped['Dst_label'])))

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3386399887.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3386399887.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Magnetic storm ratio:", (df_88_19_dropped['Dst_label'].value_counts()[1]/np.sum(df_88_19_dropped['Dst_label'].value_counts())*100).round(2), "%")


Magnetic storm?
 False - no
 True - yes
 Dst_label
False    61921
True     60840
Name: count, dtype: int64
Magnetic storm ratio: 49.56 %
Total number of elements in training dataset: 122761


In [42]:
# GAUSSIAN NAIVE BAYES

# Assuming df_88_19_dropped and df_20 are your training and testing DataFrames respectively
predictors = ["Bx", "By", "Bz", "density", "V"]

# Training data
X_train = df_88_19_dropped[predictors]
y_train = df_88_19_dropped['Dst_label'] 

# Testing data
X_test = df_20[predictors]
y_test = df_20['Dst_label']  

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of elements in training subset:", X_train.shape[0])
print("Number of elements in testing subset:", X_test.shape[0])
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
percent_error = (y_test != y_pred).sum() / X_test.shape[0]
print("The percentage of mislabeled points is:", percent_error.round(4) * 100, '%')

Number of elements in training subset: 122761
Number of elements in testing subset: 8728
Number of mislabeled points out of a total 8728 points : 1005
The percentage of mislabeled points is: 11.51 %


In [43]:
print('Confusion matrix for testing sample')

classificationSummary(y_test, gnb.predict(X_test),
                     class_names=gnb.classes_)

print('\nConfusion matrix for training sample')

classificationSummary(y_train, gnb.predict(X_train),
                     class_names=gnb.classes_)

Confusion matrix for testing sample
Confusion Matrix (Accuracy 0.8849)

       Prediction
Actual False  True
 False  7337   497
  True   508   386

Confusion matrix for training sample
Confusion Matrix (Accuracy 0.7363)

       Prediction
Actual False  True
 False 49677 12244
  True 20133 40707


In [44]:
fpr, tpr, thresholds = roc_curve(y_train, gnb.predict_proba(X_train)[:, 0], # create an Reciever Operating Curve (ROC) for the Gaussian Naive Bayes.
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr}) # Create a dataframe with values for recall and specificity.

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False) # plot the ROC.
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()
plt.savefig('gaussian_.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3214368330.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [45]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y_test], gnb.predict_proba(X_test)[:, 0]).round(3)) # Calculate the area under the ROC.

Area under the ROC curve: 0.783


In [46]:
# LINEAR DISCRIMINANT ANALYSIS.

predictors = ["Bx", "By", "Bz", "density", "V"]

# Training data
X_train = df_88_19_dropped[predictors]
y_train = df_88_19_dropped['Dst_label'] 

# Testing data
X_test = df_20[predictors]
y_test = df_20['Dst_label']  

## get the predicted label from the linear discriminant.
## Use the test sample for using the model:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Print the scaling factors by which variable data will be multiplied for better performance.
# These scaling factors are learned from the data during the training phase of the LDA model.
print(pd.DataFrame(lda.scalings_, index = X_train.columns)) 

                0
Bx       0.009174
By       0.001204
Bz      -0.146760
density  0.002102
V        0.009397


In [47]:
pred = pd.DataFrame(lda.predict_proba(X_train[predictors]), # create dataframe with LDA calculated predictions for the training dataset.
                   columns=lda.classes_)
print(pred.head())

      False     True 
0  0.906325  0.093675
1  0.785426  0.214574
2  0.705508  0.294492
3  0.935743  0.064257
4  0.752473  0.247527


In [48]:
print('Confusion matrix for testing sample')

classificationSummary(y_test, lda.predict(X_test),
                     class_names=lda.classes_)

print('\nConfusion matrix for training sample')

classificationSummary(y_train, lda.predict(X_train),
                     class_names=lda.classes_)

Confusion matrix for testing sample
Confusion Matrix (Accuracy 0.8473)

       Prediction
Actual False  True
 False  6954   880
  True   453   441

Confusion matrix for training sample
Confusion Matrix (Accuracy 0.7248)

       Prediction
Actual False  True
 False 48520 13401
  True 20386 40454


In [49]:
fpr, tpr, thresholds = roc_curve(y_train, lda.predict_proba(X_train)[:, 0], # create an Reciever Operating Curve (ROC) for the Linear Discriminant Analysis.
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr}) # Create a dataframe with values for recall and specificity.

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False) # plot the ROC.
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()
plt.savefig('lda_roc.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/3627977542.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [50]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y_test], lda.predict_proba(X_test)[:, 0]).round(3))

Area under the ROC curve: 0.758


In [51]:
# LOGISTIC REGRESSION

X = pd.get_dummies(X_train[predictors], prefix='', prefix_sep='', # Each variable is converted in as many 0/1 variables as there are different values.
                  drop_first = True, dtype = 'int') 
y = y_train

X_test_ = pd.get_dummies(X_test[predictors], prefix='', prefix_sep='', # Each variable is converted in as many 0/1 variables as there are different values.
                  drop_first = True, dtype = 'int')
y_test_ = y_test

logit_reg = LogisticRegression(penalty='l2', solver = 'liblinear') # l2 penalty uses the sum of the squares of the parameters, enables smaller but non-zero coefficients.
logit_reg.fit(X, y) # Fit the logistic regression.

print('intercept', logit_reg.intercept_[0])
print('classes', logit_reg.classes_)
pd.DataFrame({'coeff': logit_reg.coef_[0]}, 
            index = X.columns)

intercept 1.973069066415386
classes [False  True]


,coeff
-40.4,0.151263
-40.1,0.148670
-40.0,0.064125
-39.7,0.006097
-39.5,0.072656
...,...
1090,0.146988
1105,0.168421
1107,0.066079
1127,0.217782


In [52]:
# Create Confusion Matrix
classificationSummary(y, logit_reg.predict(X),
                     class_names=logit_reg.classes_)

Confusion Matrix (Accuracy 0.7552)

       Prediction
Actual False  True
 False 46459 15462
  True 14584 46256


In [53]:
fpr, tpr, thresholds = roc_curve(y, logit_reg.predict_proba(X)[:, 0],
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr})

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False)
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()  

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_4313/857124716.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [54]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y], logit_reg.predict_proba(X)[:, 0]).round(3))

Area under the ROC curve: 0.838
